# Set up

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [7]:
import os
import sys

import lightning as L
import numpy as np
import pandas as pd
import torch
from dotenv import load_dotenv
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.loggers import MLFlowLogger
from loguru import logger
from mlflow.models.signature import infer_signature
from pydantic import BaseModel
from torch.utils.data import DataLoader

import mlflow

load_dotenv()

sys.path.insert(0, "..")

from src.dataset import UserItemBinaryDFDataset as UserItemRatingDFDataset
from src.id_mapper import IDMapper
from src.sequence.inference import SequenceModelWrapper
from src.sequence.model import SequenceModel
from src.sequence.trainer import LitSequence
from src.sequence.utils import generate_item_sequences

import warnings
warnings.filterwarnings("ignore")

# Controller

In [ ]:
max_epochs = 1

In [9]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "recsys"
    run_name: str = "002-sequence"
    notebook_persist_dp: str = None
    random_seed: int = 41
    device: str = None

    batch_size: int = 128

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    embedding_dim: int = 128
    dropout: float = 0.3
    early_stopping_patience: int = 5
    learning_rate: float = 0.001
    l2_reg: float = 1e-5

    mlf_item2vec_model_name: str = "item2vec"
    mlf_model_name: str = "sequence"
    min_roc_auc: float = 0.7

    best_checkpoint_path: str = None

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        os.makedirs(self.notebook_persist_dp, exist_ok=True)

        if not (mlflow_uri := os.environ.get("MLFLOW_TRACKING_URI")):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )
            self._mlf_logger = MLFlowLogger(
                experiment_name=self.experiment_name,
                run_name=self.run_name,
                tracking_uri=mlflow_uri,
                log_model=True,
            )

        if self.device is None:
            self.device = (
                "cuda"
                if torch.cuda.is_available()
                else "mps" if torch.backends.mps.is_available() else "cpu"
            )

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2025-11-11 18:01:53.426 | INFO     | __main__:init:43 - Setting up MLflow experiment recsys - run 002-sequence...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "recsys",
  "run_name": "002-sequence",
  "notebook_persist_dp": "/mnt/d/projects/recsys/notebooks/data/002-sequence",
  "random_seed": 41,
  "device": "cuda",
  "batch_size": 128,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "embedding_dim": 128,
  "dropout": 0.3,
  "early_stopping_patience": 5,
  "learning_rate": 0.001,
  "l2_reg": 0.00001,
  "mlf_item2vec_model_name": "item2vec",
  "mlf_model_name": "sequence",
  "min_roc_auc": 0.7,
  "best_checkpoint_path": null
}


# Implement

In [15]:
def init_model(n_users, n_items, embedding_dim, dropout, item_embedding=None):
    model = SequenceModel(
        n_users, n_items, embedding_dim, dropout=dropout, item_embedding=item_embedding
    )
    return model

# Pretrain item2vec Embeddings

In [12]:
mlf_client = mlflow.MlflowClient()
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{args.mlf_item2vec_model_name}@champion"
)
skipgram_model = model.unwrap_python_model().model
embedding_0 = skipgram_model.embeddings(torch.tensor(0))
embedding_dim = embedding_0.size()[0]
id_mapping = model.unwrap_python_model().id_mapping
pretrained_item_embedding = skipgram_model.embeddings

In [13]:
assert (
    pretrained_item_embedding.embedding_dim == args.embedding_dim
), "Mismatch pretrained item_embedding dimension"

# Test implementation

In [44]:
embedding_dim = 8
batch_size = 2

# Mock data
user_indices = [0, 0, 1, 2, 2]
item_indices = [0, 1, 2, 3, 4]
timestamps = [0, 1, 2, 3, 4]
ratings = [0, 4, 5, 3, 0]
item_sequences = [
    [-1, -1, 2, 3],
    [-1, -1, 2, 3],
    [-1, -1, 1, 3],
    [-1, -1, 2, 1],
    [-1, -1, 2, 1],
]

n_users = len(set(user_indices))
n_items = len(set(item_indices))

train_df = pd.DataFrame(
    {
        "user_indice": user_indices,
        "item_indice": item_indices,
        args.timestamp_col: timestamps,
        args.rating_col: ratings,
        "item_sequence": item_sequences,
    }
)

model = init_model(n_users, n_items, embedding_dim, args.dropout)

# Example forward pass
model.eval()
user = torch.tensor([0])
item_sequence = torch.tensor([[-1, -1, -1, 0, 1]])
target_item = torch.tensor([2])
predictions = model.predict(user, item_sequence, target_item)
print(predictions)
model.train()

tensor([[0.5193]], grad_fn=<SigmoidBackward0>)


SequenceModel(
  (item_embedding): Embedding(6, 8, padding_idx=5)
  (user_embeddings): Embedding(3, 8)
  (gru): GRU(8, 8, batch_first=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Sequential(
    (0): Linear(in_features=24, out_features=8, bias=True)
    (1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=8, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

In [47]:
model.gru.training

True

In [46]:
rating_dataset = UserItemRatingDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col
)

train_loader = DataLoader(
    rating_dataset, batch_size=batch_size, shuffle=False, drop_last=True
)

In [27]:
for batch_input in train_loader:
    print(batch_input)

{'user': tensor([0, 0]), 'item': tensor([0, 1]), 'rating': tensor([0., 1.]), 'item_sequence': tensor([[-1, -1,  2,  3],
        [-1, -1,  2,  3]]), 'item_sequence_ts_bucket': tensor([], size=(2, 0), dtype=torch.int64), 'item_feature': tensor([], size=(2, 0))}
{'user': tensor([1, 2]), 'item': tensor([2, 3]), 'rating': tensor([1., 1.]), 'item_sequence': tensor([[-1, -1,  1,  3],
        [-1, -1,  2,  1]]), 'item_sequence_ts_bucket': tensor([], size=(2, 0), dtype=torch.int64), 'item_feature': tensor([], size=(2, 0))}


In [48]:
# model
lit_model = LitSequence(model, log_dir=args.notebook_persist_dp)

# train model
trainer = L.Trainer(
    default_root_dir=f"{args.notebook_persist_dp}/test",
    max_epochs=2,
    accelerator=args.device if args.device else "auto",
)
trainer.fit(
    model=lit_model, train_dataloaders=train_loader, val_dataloaders=train_loader
)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type          | Params | Mode 
-------------------------------------------------------------
0 | sequence_model     | SequenceModel | 729    | train
1 | val_roc_auc_metric | BinaryAUROC   | 0      | train
-------------------------------------------------------------
729       Trainable params
0         Non-trainable params
729       Total params
0.003     Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                     | 0/? [00:00<?, ?it/s]

Training: |                                                            | 0/? [00:00<?, ?it/s]

Validation: |                                                          | 0/? [00:00<?, ?it/s]

Validation: |                                                          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.
2025-11-11 20:03:34.823 | INFO     | src.sequence.trainer:on_fit_end:110 - Logging classification metrics...


In [49]:
users = torch.tensor([0, 0, 0, 0])
item_sequences = torch.tensor(
    [[-1, -1, 2, 3], [-1, -1, 2, 3], [-1, -1, 1, 3], [-1, -1, 2, 1]]
)
items = torch.tensor([0, 1, 2, 3])
predictions = model.predict(users, item_sequences, items)
print(predictions)

tensor([[0.5316],
        [0.5818],
        [0.5067],
        [0.5136]], grad_fn=<SigmoidBackward0>)


In [50]:
def create_predict_df(
    train_df,
    val_user_indices,
    val_timestamp,
    rating_col,
    timestamp_col,
    sequence_length=10,
):
    predict_df = pd.DataFrame(
        {
            "user_indice": val_user_indices,
            "item_indice": -1,  # placeholder
            "timestamp": val_timestamp,
            "source": "predict",
        }
    )

    predict_df = (
        pd.concat(
            [
                train_df.loc[lambda df: df[rating_col].gt(0)][
                    ["user_indice", "item_indice", timestamp_col]
                ].assign(source="train"),
                predict_df,
            ],
            axis=0,
        )
        .pipe(
            generate_item_sequences,
            "user_indice",
            "item_indice",
            timestamp_col,
            sequence_length=sequence_length,
            padding=True,
            padding_value=-1,
        )
        .loc[lambda df: df["source"].eq("predict")]
        .assign(item_sequence=lambda df: df["item_sequence"].apply(np.array))
    )

    return predict_df


predict_df = create_predict_df(
    train_df,
    user_indices,
    timestamps[-1],
    args.rating_col,
    args.timestamp_col,
    sequence_length=10,
)

predict_df

,user_indice,item_indice,timestamp,source,item_sequence
0,0,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 1]"
1,0,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, 1, -1]"
2,1,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 2]"
3,2,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, 3]"
4,2,-1,4,predict,"[-1, -1, -1, -1, -1, -1, -1, -1, 3, -1]"


In [51]:
recommendations = model.recommend(
    torch.tensor(predict_df["user_indice"].values),
    torch.tensor(predict_df["item_sequence"].values.tolist()),
    k=2,
    batch_size=4,
)
recommendations

Generating recommendations:   0%|          | 0/2 [00:00<?, ?it/s]

{'user_indices': [0, 0, 0, 0, 1, 1, 2, 2, 2, 2],
 'recommendation': [1, 4, 1, 4, 2, 1, 1, 4, 1, 4],
 'score': [0.5900562405586243,
  0.5514488816261292,
  0.5886162519454956,
  0.5502788424491882,
  0.5293713808059692,
  0.5201959013938904,
  0.46934831142425537,
  0.46253883838653564,
  0.4743664860725403,
  0.4699535071849823]}

# Prep data

In [52]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm_fp = "../data/idm.json"
idm = IDMapper().load(idm_fp)

assert (
    train_df[args.user_col].map(lambda s: idm.get_user_index(s))
    != train_df["user_indice"]
).sum() == 0, "Mismatch IDM"
assert (
    val_df[args.user_col].map(lambda s: idm.get_user_index(s)) != val_df["user_indice"]
).sum() == 0, "Mismatch IDM"

In [53]:
user_indices = train_df["user_indice"].unique()
item_indices = train_df["item_indice"].unique()

logger.info(f"{len(user_indices)=:,.0f}, {len(item_indices)=:,.0f}")

2025-11-11 20:05:39.535 | INFO     | __main__:<module>:4 - len(user_indices)=19,578, len(item_indices)=4,630


In [54]:
train_df

,user_id,parent_asin,rating,timestamp,timestamp_unix,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,...,user_rating_list_10_recent_asin,user_rating_list_10_recent_asin_timestamp,item_sequence,item_sequence_ts,item_sequence_ts_bucket,main_category,title,description,categories,price
0,AEGX2IOMXXZWDP5LJJVPFARC2LXA,B0014CN2H6,0.0,2015-02-13 16:41:46.000,NaN,1.0,5.000000,0.0,NaN,0.0,...,"B001QCWSLU,B07WWJDWY3,B0077LRPSA,B000U34SZA,B0...","1353957492,1353957582,1353957737,1353957797,13...","[1143, 4229, 2045, 596, 705, 2616, 2232, 2169,...","[1353957492, 1353957582, 1353957737, 135395779...","[6, 6, 6, 6, 6, 6, 6, 6, 0, 0]",Video Games,Shin Megami Tensei: Persona 3 FES - PlayStation 2,[A brand new chapter of Persona 3 featuring 30...,"[Video Games, Legacy Systems, PlayStation Syst...",37.99
1,AEEEFJCFELMIUQM7S4MN3OJXBE6Q,B0BM5YPLKM,0.0,2008-12-21 01:53:58.000,NaN,NaN,NaN,NaN,NaN,NaN,...,"B001EYUXXU,B09MFBJJQJ,B001EYUXXK,B00004S9AF,B0...","1182172867,1211165279,1211422243,1221445917,12...","[-1, -1, -1, -1, -1, 1032, 4491, 1031, 85, 554]","[-1, -1, -1, -1, -1, 1182172867, 1211165279, 1...","[-1, -1, -1, -1, -1, 6, 5, 5, 5, 5]",Computers,"Classic N64 Controller, SAFFUN N64 Wired USB P...",[Customers Frequently Asked Questions: Q: 1. W...,"[Video Games, PC, Accessories, Controllers, Jo...",17.99
2,AEKGUEYKOTY2NDAZNTRIHXCQBVXQ,B009DL2TBA,0.0,2016-01-30 18:08:13.000,NaN,29.0,4.586207,5.0,4.4,1.0,...,"B005ZBNUUG,B093GLPLLB","1404320532,1444061695","[-1, -1, -1, -1, -1, -1, -1, -1, 1965, 4452]","[-1, -1, -1, -1, -1, -1, -1, -1, 1404320532, 1...","[-1, -1, -1, -1, -1, -1, -1, -1, 6, 5]",Video Games,PlayStation 3 500 GB System,"[Play, Stream, and Watch your favorite games, ...","[Video Games, Legacy Systems, PlayStation Syst...",233.98
3,AFJO7KXBEW73L2UN7MMKPVHLWORQ,B00006LELI,4.0,2003-11-04 00:26:02.000,1.067906e+09,12.0,4.500000,1.0,4.0,0.0,...,None,None,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",Video Games,Resident Evil Zero - Gamecube,"[Product Description, See how it all began. Th...","[Video Games, Legacy Systems, Nintendo Systems...",29.98
4,AEZC3FNIGVAQMZT7T2MNXZZOPQZQ,B002BSA2LQ,5.0,2021-05-31 01:24:23.280,1.622424e+09,0.0,NaN,0.0,NaN,0.0,...,"B000UODATY,B001ELJFPA,B07VLCRZ21,B00308JV6I,B0...","1622423201,1622423973,1622423990,1622424022,16...","[604, 808, 4207, 1405, 1583, 1018, 1489, 1446,...","[1622423201, 1622423973, 1622423990, 162242402...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Video Games,Forza Motorsport 3 - Xbox 360,"[Product Description, Whether it’s an exotic s...","[Video Games, Legacy Systems, Xbox Systems, Xb...",31.49
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
328591,AG5K6I36DZU2YYIZOU7F3PNPNQOQ,B07BNKWG2D,0.0,2015-12-27 19:35:23.000,NaN,NaN,NaN,NaN,NaN,NaN,...,"B003O6FV8S,B004XV6ST4,B00ZHQ39F0,B001G60JHQ","1297130057,1369331267,1451244466,1451244811","[-1, -1, -1, -1, -1, -1, 1518, 1794, 3068, 1067]","[-1, -1, -1, -1, -1, -1, 1297130057, 136933126...","[-1, -1, -1, -1, -1, -1, 7, 6, 0, 0]",Video Games,LEGO Disney Pixar's The Incredibles - Nintendo...,[Experience the thrilling adventures of the Pa...,"[Video Games, Nintendo Switch, Games]",None
328592,AFCSNRZGIHCXTCTADIEW7SY5BRPQ,B076ZPW7R2,0.0,2013-07-25 18:58:01.000,NaN,NaN,NaN,NaN,NaN,NaN,...,"B0030HM2KQ,B0080CAOYM,B00COAGPG2","1361732871,1368381197,1372209466","[-1, -1, -1, -1, -1, -1, -1, 1409, 2102, 2401]","[-1, -1, -1, -1, -1, -1, -1, 1361732871, 13683...","[-1, -1, -1, -1, -1, -1, -1, 5, 5, 4]",Video Games,Senran Kagura Estival Versus - PlayStation 4,"[The high-flying, high-bouncing ninja girls of...","[Video Games, PlayStation 4, Games]",119.0
328593,AGSDWRVEL7YOJTHZLFKW6IIEPE3Q,B00BCX2AZW,0.0,2009-01-30 04:16:53.000,NaN,NaN,NaN,NaN,NaN,NaN,...,"B00008URUA,B00005NN5G,B0001VNNE8,B001EYUWBI,B0...","1167923982,1169856191,1169

# Train

In [55]:
rating_dataset = UserItemRatingDFDataset(
    train_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col
)
val_rating_dataset = UserItemRatingDFDataset(
    val_df, "user_indice", "item_indice", args.rating_col, args.timestamp_col
)

train_loader = DataLoader(
    rating_dataset, batch_size=args.batch_size, shuffle=True, drop_last=True
)
val_loader = DataLoader(
    val_rating_dataset, batch_size=args.batch_size, shuffle=False, drop_last=False
)

In [56]:
n_items = len(item_indices)
n_users = len(user_indices)

model = init_model(n_users, n_items, args.embedding_dim, args.dropout)

#### Predict before train

In [57]:
model.item_embedding

Embedding(4631, 128, padding_idx=4630)

In [58]:
val_df = val_rating_dataset.df
val_df.sample(5)

,user_id,parent_asin,rating,timestamp,timestamp_unix,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,...,user_rating_list_10_recent_asin,user_rating_list_10_recent_asin_timestamp,item_sequence,item_sequence_ts,item_sequence_ts_bucket,main_category,title,description,categories,price
1554,AG2KBJG5DMEIISPJVF3OVMRB4ALA,B09CG15F86,1.0,2021-11-14 03:43:00.056,1.636861e+09,4.0,4.500000,1.0,4.0,0.0,...,"B0055464UQ,B0719W8QL3,B01N3ASPNV,B072V478NR,B0...","1508645444,1555058022,1555058205,1555059588,15...","[-1, -1, -1, -1, -1, 1878, 3632, 3527, 3705, 4...","[-1, -1, -1, -1, -1, 1508645444, 1555058022, 1...","[-1, -1, -1, -1, -1, 7, 6, 6, 6, 6]",Computers,Razer Doubleshot PBT Keycap Upgrade Set for Me...,[Enjoy durability without cramping your style....,"[Video Games, PC, Accessories, Gaming Keyboards]",None
1022,AGYJI6XSILTABA3ZWALFM6SMEVOQ,B001EYUOJS,0.0,2021-12-10 03:47:10.046,NaN,1.0,5.000000,0.0,NaN,0.0,...,"B07SM7G9CN,B07G1SC6BW,B07L6MJ6LD,B07J3P1GJM,B0...","1527019952,1538858187,1557937823,1557937962,16...","[-1, -1, 4164, 3970, 4049, 4018, 4388, 3697, 3...","[-1, -1, 1527019952, 1538858187, 1557937823, 1...","[-1, -1, 7, 7, 6, 6, 6, 6, 4, 4]",Video Games,Time Splitters 2 - PlayStation 2,"[Product Description, The evil race of TimeSpl...","[Video Games, Legacy Systems, PlayStation Syst...",99.99
1913,AEL2FTDUA5KWOWBLN2HXG3CJRFRA,B08DHTZNNF,1.0,2021-09-02 13:28:34.010,1.630589e+09,13.0,3.769231,1.0,5.0,0.0,...,"B00RJNE3TK,B018K6KV68,B09JDLC31H,B087NN2K41,B0...","1462572545,1471776059,1491916082,1493417579,14...","[-1, -1, -1, -1, 2922, 3223, 4479, 4341, 4497,...","[-1, -1, -1, -1, 1462572545, 1471776059, 14919...","[-1, -1, -1, -1, 8, 8, 7, 7, 7, 5]",Computers,"Razer Mamba Elite Wired Gaming Mouse: 16,000 D...",[The Razer Mamba Elite features our acclaimed ...,"[Video Games, Legacy Systems, PlayStation Syst...",36.95
793,AF4VS3TYUZMQGRDDUJOFWMQXAL3Q,B07BFWLXLS,1.0,2022-01-08 21:25:37.898,1.641677e+09,2.0,5.000000,1.0,5.0,0.0,...,"B087SLFYPJ,B084QDTVC6,B087SHFL9B,B087NNPYP3,B0...","1625425418,1625425746,1625425963,1626296365,16...","[4345, 4327, 4344, 4342, 4375, 4439, 4368, 434...","[1625425418, 1625425746, 1625425963, 162629636...","[5, 5, 5, 5, 5, 5, 5, 5, 1, 0]",Video Games,Kirby Star Allies - Nintendo Switch,"[When a new evil threatens planet pop star, Ki...","[Video Games, Nintendo Switch, Games]",44.97
872,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B01CEFEMFW,0.0,2022-01-28 02:06:05.345,NaN,0.0,NaN,0.0,NaN,0.0,...,"B00004XOWT,B00005LOXE,B00000JRSB,B011ALES6E,B0...","1496369526,1496369581,1496369625,1496369716,15...","[-1, -1, -1, 114, 144, 26, 3102, 4032, 4209, 4...","[-1, -1, -1, 1496369526, 1496369581, 149636962...","[-1, -1, -1, 7, 7, 7, 7, 6, 6, 6]",Computers,Logitech G900 Chaos Spectrum Professional Grad...,"[In high-pressure moments, lag time can be the...","[Video Games, PC, Accessories, Gaming Mice]",81.99


In [59]:
user_id = val_df.sample(1)[args.user_col].values[0]
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
with pd.option_context("display.max_colwidth", None):
    display(test_df)

,user_id,parent_asin,rating,timestamp,timestamp_unix,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,...,user_rating_list_10_recent_asin,user_rating_list_10_recent_asin_timestamp,item_sequence,item_sequence_ts,item_sequence_ts_bucket,main_category,title,description,categories,price
935,AFQ4YW6MBVNAQH3ZOXJPDVGFR2RA,B0C3KYVDWT,0.0,2022-01-28 01:55:18.351,NaN,35.0,4.942857,2.0,5.0,0.0,...,"B00503E9FY,B07X1HF3V6,B003BFW4OG,B0C6BV5XQZ,B00HLUZXAY,B00Z9TM72Q,B07WS18ZS3,B01GY35W22","1333279768,1362440671,1362440794,1375108160,1439745327,1493563003,1493563098,1563740421","[-1, -1, 1807, 4237, 1447, 4615, 2602, 3059, 4217, 3377]","[-1, -1, 1333279768, 1362440671, 1362440794, 1375108160, 1439745327, 1493563003, 1493563098, 1563740421]","[-1, -1, 8, 8, 8, 8, 8, 7, 7, 6]",Computers,"SanDisk 128GB microSDXC-Card, Licensed for Nintendo-Switch - SDSQXAO-128G-GNCZN","[With incredible speed, the officially licensed SanDisk microSDXC card for the Nintendo Switch lets you add up to [64/128/256GB] of space for your system. Spend less time waiting and more time gaming with read and write speeds of up to 100MB/s and 90MB/s respectively. Perfect for keeping your favorite games in one place. Backed by a lifetime limited warranty, so you’re ready for the long haul. 1GB=1,000,000,000 bytes. Actual user storage less. (1)128-256GB: Up to 100MB/s read speed; Up to 90MB/s write speed. 64GB: Up to 100MB/s read speed; Up to 60MB/s write speed. Based on internal testing; performance may be lower depending upon host device interface, usage conditions and other factors. 1MB = 1,000,000 bytes. (2) 30 year warranty in Germany, Canada, and regions not recognizing lifetime warranty.]","[Video Games, Nintendo Switch, Accessories]",14.99
1712,AFQ4YW6MBVNAQH3ZOXJPDVGFR2RA,B07V8YSBFG,1.0,2022-01-28 01:55:18.351,1.643335e+09,10.0,4.600000,1.0,1.0,0.0,...,"B00503E9FY,B07X1HF3V6,B003BFW4OG,B0C6BV5XQZ,B00HLUZXAY,B00Z9TM72Q,B07WS18ZS3,B01GY35W22","1333279768,1362440671,1362440794,1375108160,1439745327,1493563003,1493563098,1563740421","[-1, -1, 1807, 4237, 1447, 4615, 2602, 3059, 4217, 3377]","[-1, -1, 1333279768, 1362440671, 1362440794, 1375108160, 1439745327, 1493563003, 1493563098, 1563740421]","[-1, -1, 8, 8, 8, 8, 8, 7, 7, 6]",Video Games,"Roblox Digital Gift Code for 1,200 Robux [Redeem Worldwide - Includes Exclusive Virtual Item] [Online Game Code]",[],"[Video Games, Digital for the Holidays, Digital Video Games]",15.0


In [60]:
test_row = test_df.loc[lambda df: df[args.rating_col].gt(0)].iloc[0]
item_id = test_row[args.item_col]
item_sequence = test_row["item_sequence"]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)
user = torch.tensor([user_indice])
item_sequence = torch.tensor([item_sequence])
item = torch.tensor([item_indice])

model.eval()
model.predict(user, item_sequence, item)
model.train()

2025-11-11 20:07:01.905 | INFO     | __main__:<module>:4 - Test predicting before training with user_id = AFQ4YW6MBVNAQH3ZOXJPDVGFR2RA and parent_asin = B07V8YSBFG


SequenceModel(
  (item_embedding): Embedding(4631, 128, padding_idx=4630)
  (user_embeddings): Embedding(19578, 128)
  (gru): GRU(128, 128, batch_first=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc): Sequential(
    (0): Linear(in_features=384, out_features=128, bias=True)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=1, bias=True)
    (5): Sigmoid()
  )
)

#### Training loop

##### Overfit 1 batch

In [77]:
early_stopping = EarlyStopping(
    monitor="val_roc_auc", patience=10, mode="max", verbose=False
)

model = init_model(n_users, n_items, args.embedding_dim, dropout=0)
lit_model = LitSequence(
    model,
    learning_rate=args.learning_rate,
    l2_reg=0.0,
    log_dir=args.notebook_persist_dp,
    accelerator=args.device,
)

log_dir = f"{args.notebook_persist_dp}/logs/overfit"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    accelerator=args.device if args.device else "auto",
    max_epochs=100,
    overfit_batches=1,
    callbacks=[early_stopping],
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=train_loader,
)
logger.info(f"Logs available at {trainer.log_dir}")

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
`Trainer(overfit_batches=1)` was configured so 1 batch will be used.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type          | Params | Mode 
-------------------------------------------------------------
0 | sequence_model     | SequenceModel | 3.2 M  | train
1 | val_roc_auc_metric | BinaryAUROC   | 0      | train
-------------------------------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.990    Total estimated model params size (MB)
12        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                     | 0/? [00:00<?, ?it/s]

Training: |                                                            | 0/? [00:00<?, ?it/s]

Validation: |                                                          | 0/? [00:00<?, ?it/s]

Validation: |                                                          | 0/? [00:00<?, ?it/s]

...

Validation: |                                                          | 0/? [00:00<?, ?it/s]

Validation: |                                                          | 0/? [00:00<?, ?it/s]

Validation: |                                                          | 0/? [00:00<?, ?it/s]

Validation: |                                                          | 0/? [00:00<?, ?it/s]

2025-11-11 20:51:47.332 | INFO     | src.sequence.trainer:on_fit_end:12 - Logging classification metrics...
2025-11-11 20:52:51.226 | INFO     | __main__:<module>:29 - Logs available at /mnt/d/projects/recsys/notebooks/data/002-sequence/logs/overfit/lightning_logs/version_3


In [78]:
%tensorboard --logdir $trainer.log_dir

##### Fit on all data

In [79]:
max_epochs

1

In [80]:
train_loader.dataset.df.head()

,user_id,parent_asin,rating,timestamp,timestamp_unix,parent_asin_rating_cnt_365d,parent_asin_rating_avg_prev_rating_365d,parent_asin_rating_cnt_90d,parent_asin_rating_avg_prev_rating_90d,parent_asin_rating_cnt_30d,...,user_rating_list_10_recent_asin,user_rating_list_10_recent_asin_timestamp,item_sequence,item_sequence_ts,item_sequence_ts_bucket,main_category,title,description,categories,price
0,AEGX2IOMXXZWDP5LJJVPFARC2LXA,B0014CN2H6,0.0,2015-02-13 16:41:46.000,NaN,1.0,5.000000,0.0,NaN,0.0,...,"B001QCWSLU,B07WWJDWY3,B0077LRPSA,B000U34SZA,B0...","1353957492,1353957582,1353957737,1353957797,13...","[1143, 4229, 2045, 596, 705, 2616, 2232, 2169,...","[1353957492, 1353957582, 1353957737, 135395779...","[6, 6, 6, 6, 6, 6, 6, 6, 0, 0]",Video Games,Shin Megami Tensei: Persona 3 FES - PlayStation 2,[A brand new chapter of Persona 3 featuring 30...,"[Video Games, Legacy Systems, PlayStation Syst...",37.99
1,AEEEFJCFELMIUQM7S4MN3OJXBE6Q,B0BM5YPLKM,0.0,2008-12-21 01:53:58.000,NaN,NaN,NaN,NaN,NaN,NaN,...,"B001EYUXXU,B09MFBJJQJ,B001EYUXXK,B00004S9AF,B0...","1182172867,1211165279,1211422243,1221445917,12...","[-1, -1, -1, -1, -1, 1032, 4491, 1031, 85, 554]","[-1, -1, -1, -1, -1, 1182172867, 1211165279, 1...","[-1, -1, -1, -1, -1, 6, 5, 5, 5, 5]",Computers,"Classic N64 Controller, SAFFUN N64 Wired USB P...",[Customers Frequently Asked Questions: Q: 1. W...,"[Video Games, PC, Accessories, Controllers, Jo...",17.99
2,AEKGUEYKOTY2NDAZNTRIHXCQBVXQ,B009DL2TBA,0.0,2016-01-30 18:08:13.000,NaN,29.0,4.586207,5.0,4.4,1.0,...,"B005ZBNUUG,B093GLPLLB","1404320532,1444061695","[-1, -1, -1, -1, -1, -1, -1, -1, 1965, 4452]","[-1, -1, -1, -1, -1, -1, -1, -1, 1404320532, 1...","[-1, -1, -1, -1, -1, -1, -1, -1, 6, 5]",Video Games,PlayStation 3 500 GB System,"[Play, Stream, and Watch your favorite games, ...","[Video Games, Legacy Systems, PlayStation Syst...",233.98
3,AFJO7KXBEW73L2UN7MMKPVHLWORQ,B00006LELI,1.0,2003-11-04 00:26:02.000,1.067906e+09,12.0,4.500000,1.0,4.0,0.0,...,None,None,"[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]","[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1]",Video Games,Resident Evil Zero - Gamecube,"[Product Description, See how it all began. Th...","[Video Games, Legacy Systems, Nintendo Systems...",29.98
4,AEZC3FNIGVAQMZT7T2MNXZZOPQZQ,B002BSA2LQ,1.0,2021-05-31 01:24:23.280,1.622424e+09,0.0,NaN,0.0,NaN,0.0,...,"B000UODATY,B001ELJFPA,B07VLCRZ21,B00308JV6I,B0...","1622423201,1622423973,1622423990,1622424022,16...","[604, 808, 4207, 1405, 1583, 1018, 1489, 1446,...","[1622423201, 1622423973, 1622423990, 162242402...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",Video Games,Forza Motorsport 3 - Xbox 360,"[Product Description, Whether it’s an exotic s...","[Video Games, Legacy Systems, Xbox Systems, Xb...",31.49


In [83]:
# papermill_description=fit-model
early_stopping = EarlyStopping(
    monitor="val_roc_auc", patience=args.early_stopping_patience, mode="max", verbose=False
)

checkpoint_callback = ModelCheckpoint(
    dirpath=f"{args.notebook_persist_dp}/checkpoints",
    filename="best-checkpoint",
    save_top_k=1,
    monitor="val_roc_auc",
    mode="max",
)

model = init_model(
    n_users,
    n_items,
    args.embedding_dim,
    dropout=args.dropout,
    item_embedding=pretrained_item_embedding,
)
lit_model = LitSequence(
    model,
    learning_rate=args.learning_rate,
    l2_reg=args.l2_reg,
    log_dir=args.notebook_persist_dp,
    evaluate_ranking=True,
    idm=idm,
    args=args,
    accelerator=args.device,
    checkpoint_callback=checkpoint_callback,
)

log_dir = f"{args.notebook_persist_dp}/logs/run"

# train model
trainer = L.Trainer(
    default_root_dir=log_dir,
    max_epochs=max_epochs,
    callbacks=[early_stopping, checkpoint_callback],
    accelerator=args.device if args.device else "auto",
    logger=args._mlf_logger if args.log_to_mlflow else None,
)
trainer.fit(
    model=lit_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader,
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name               | Type          | Params | Mode 
-------------------------------------------------------------
0 | sequence_model     | SequenceModel | 3.2 M  | train
1 | val_roc_auc_metric | BinaryAUROC   | 0      | train
-------------------------------------------------------------
3.2 M     Trainable params
0         Non-trainable params
3.2 M     Total params
12.990    Total estimated model params size (MB)
11        Modules in train mode
1         Modules in eval mode


Sanity Checking: |                                                     | 0/? [00:00<?, ?it/s]

Training: |                                                            | 0/? [00:00<?, ?it/s]

Validation: |                                                          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.
2025-11-11 21:12:34.950 | INFO     | src.sequence.trainer:on_fit_end:4 - Loading best model from /mnt/d/projects/recsys/notebooks/data/002-sequence/checkpoints/best-checkpoint-v8.ckpt...
2025-11-11 21:12:35.357 | INFO     | src.sequence.trainer:on_fit_end:12 - Logging classification metrics...
2025-11-11 21:12:40.469 | INFO     | src.sequence.trainer:on_fit_end:15 - Logging ranking metrics...


Generating recommendations:   0%|          | 0/177 [00:00<?, ?it/s]

🏃 View run 002-sequence at: http://localhost:5002/#/experiments/1/runs/06d06d4101dd4806a4cbf0ce724bae7b
🧪 View experiment at: http://localhost:5002/#/experiments/1


In [85]:
logger.info(
    f"Test predicting after training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
model.eval()
model = model.to(user.device)
model.predict(user, item_sequence, item)

2025-11-11 21:25:48.817 | INFO     | __main__:<module>:1 - Test predicting after training with user_id = AFQ4YW6MBVNAQH3ZOXJPDVGFR2RA and parent_asin = B07V8YSBFG


tensor([[0.7523]], grad_fn=<SigmoidBackward0>)

# Load best checkpoint

In [ ]:
logger.info(f"Loading best checkpoint from {checkpoint_callback.best_model_path}...")
args.best_checkpoint_path = checkpoint_callback.best_model_path

best_trainer = LitSequence.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    model=init_model(n_users, n_items, args.embedding_dim, dropout=0),
)

2025-11-11 21:38:32.955 | INFO     | __main__:<module>:1 - Loading best checkpoint from /mnt/d/projects/recsys/notebooks/data/002-sequence/checkpoints/best-checkpoint-v8.ckpt...


In [ ]:
best_model = best_trainer.model.to(lit_model.device)

In [91]:
best_model.eval()
best_model.predict(user, item_sequence, item)

tensor([[0.7523]], grad_fn=<SigmoidBackward0>)

### Persist id mapping

In [92]:
if args.log_to_mlflow:
    # Persist id_mapping so that at inference we can predict based on item_ids (string) instead of item_index
    run_id = trainer.logger.run_id
    mlf_client = trainer.logger.experiment
    mlf_client.log_artifact(run_id, idm_fp)

### Wrap inference function and register best checkpoint as MLflow model

In [ ]:
sample_input = {
    "user_ids": [idm.get_user_id(0)],
    "item_sequences": [[idm.get_item_id(0), idm.get_item_id(1)]],
    "item_ids": [idm.get_item_id(0)],
}
sample_output = best_model(
    torch.tensor([0]),
    torch.tensor([0]),
    torch.tensor([[0, 1]])
).detach().numpy()

sample_output

array([[0.6201483]], dtype=float32)

In [102]:
idm_fp

'../data/idm.json'

In [ ]:
if args.log_to_mlflow:
    run_id = trainer.logger.run_id
    signature = infer_signature(sample_input, sample_output)

    idm_fn = idm_fp.split("/")[-1]
    with mlflow.start_run(run_id=run_id, nested=True):
        artifacts = {
            "model_path": checkpoint_callback.best_model_path,
            "id_mapping": mlflow.get_artifact_uri(idm_fn),
        }

        mlflow.pyfunc.log_model(
            artifact_path="inferrer",
            python_model=SequenceModelWrapper(),
            artifacts=artifacts,
            signature=signature,
            input_example=sample_input,
            registered_model_name=args.mlf_model_name,
        )

    print(f"Model logged to MLflow run {run_id}")


2025/11/11 21:50:09 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 21:50:10 INFO mlflow.pyfunc: Validating input example against model signature


2025/11/11 21:51:27 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpiob84dju/model, flavor: python_function). Fall back to return ['cloudpickle==3.1.2']. Set logging level to DEBUG to see the full traceback. 
2025/11/11 21:51:28 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
2025/11/11 21:51:28 WARNING mlflow.models.model: Failed to validate serving input example {
  "inputs": {
    "user_ids": [
      "AE225O22S.... Alternatively, you can avoid passing input example and pass model signature instead when logging the model. To ensure the input example is valid prior to serving, please try calling `mlflow.models.validate_serving_input` on the model uri and serving input example. A serving input example can be generated from model input example using `mlflow.models.convert_input_example_to_serving_input` functi

🏃 View run 002-sequence at: http://localhost:5002/#/experiments/1/runs/06d06d4101dd4806a4cbf0ce724bae7b
🧪 View experiment at: http://localhost:5002/#/experiments/1
Model logged to MLflow run 06d06d4101dd4806a4cbf0ce724bae7b


# Set the newly trained model as champion

In [104]:
if args.log_to_mlflow:
    val_roc_auc = trainer.logger.experiment.get_run(trainer.logger.run_id).data.metrics[
        "val_roc_auc"
    ]

    if val_roc_auc > args.min_roc_auc:
        logger.info(f"Aliasing the new model as champion...")
        model_version = (
            mlf_client.get_registered_model(args.mlf_model_name)
            .latest_versions[0]
            .version
        )

        mlf_client.set_registered_model_alias(
            name=args.mlf_model_name, alias="champion", version=model_version
        )

        mlf_client.set_model_version_tag(
            name=args.mlf_model_name,
            version=model_version,
            key="author",
            value="minh",
        )

2025-11-11 21:51:54.732 | INFO     | __main__:<module>:7 - Aliasing the new model as champion...


# Clean up

In [101]:
all_params = [args]

if args.log_to_mlflow:
    with mlflow.start_run(run_id=run_id):
        for params in all_params:
            params_dict = params.dict()
            params_ = dict()
            for k, v in params_dict.items():
                if k == "top_K":
                    k = "top_big_K"
                if k == "top_k":
                    k = "top_small_k"
                params_[f"{params.__repr_name__()}.{k}"] = v
            mlflow.log_params(params_)

🏃 View run 002-sequence at: http://localhost:5002/#/experiments/1/runs/06d06d4101dd4806a4cbf0ce724bae7b
🧪 View experiment at: http://localhost:5002/#/experiments/1
